# BioMedQuery.UMLS


Utilities to search the Unified Medical Language System (UMLS). This is a Julia interface to their [REST API](https://documentation.uts.nlm.nih.gov/rest/home.html).

Searching the UMLS requires approved credentials. You can sign up [here](https://uts.nlm.nih.gov//license.html)

As of today, the following utilities are available:

* verify credentials / issue umls tickets
* search_umls
* get the best maching cui from a query
* get the semantic type

## 1. Set Up

In [ ]:
using BioMedQuery.UMLS
user = ENV["UMLS_USER"];
psswd = ENV["UMLS_PSSWD"];
credentials = Credentials(user, psswd)
query = Dict("string"=>"asthma", "searchType"=>"exact")

## 2. Get a ticket and submit query

In [ ]:
tgt = get_tgt(credentials)
all_results = search_umls(tgt, query)

## 3.  Get best matching cui and it's semantic type

In [ ]:
cui = best_match_cui(all_results)
display(cui)
sm = get_semantic_type(tgt, cui)
display(sm)

### Processes available for UMLS

a. Get all UMLS semantic types for all MeSH sotore in a database corresponding to results from an Entrez query

b. For all articles in the 'results' database, filter all MeSH associated with a specific semantic type

In [ ]:
using BioMedQuery.Processes
using MySQL

db_host = "127.0.0.1"
mysql_usr = "root"
mysql_pswd = "bcbi123"
dbname = "biomed_query_test"

db = mysql_connect(db_host, mysql_usr, mysql_pswd, dbname)

map_mesh_to_umls_async!(db, credentials)


In [ ]:
tables = mysql_execute(db, "show tables;")
display(tables)

In [ ]:
mesh2umls = mysql_execute(db, "select * from mesh2umls")
display(mesh2umls)

### b. Filter by semantic type

In [ ]:
labels2ind, occur = umls_semantic_occurrences(db, "Disease or Syndrome")

println("-------------------------------------------------------------")
println("Output Descritor to Index Dictionary")
display(labels2ind)
println("-------------------------------------------------------------")

println("-------------------------------------------------------------")
println("Output Data Matrix")
display(full(occur))
println("-------------------------------------------------------------")


### Plot conditional probabilities as a histogram

In [ ]:
collect(keys(labels2ind))

In [ ]:
using PlotlyJS

trace1 = bar(;x=collect(keys(labels2ind)),
            y=sum(occur, 2)[:]./10,
            marker=attr(color="rgba(50, 171, 96, 0.7)",
            line=attr(color="rgba(50, 171, 96, 1.0)", width=2)))

data = [trace1]
layout = Layout(;margin_b = 100,
                 margin_r =100)

plot(data, layout)